# Example: Hyperbolic Trajectory

## Problem Statement

A geocentric trajectory has a perigee altitude of 300 km and a perigee velocity of 15 km/s. Calculate the time to fly from perigee to a true anomaly of $\theta =$ 100°, and the position at that time. Then, calculate the true anomaly and speed 3 hr later.

## Solution

The first step in these problems is always to find the primary orbital parameters, $h$ and $e$. To find $h$, we can use the given perigee data:

$$h = v_p r_p$$

In [1]:
# %matplotlib notebook
import numpy as np
from scipy.optimize import newton
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle, Arc, Rectangle

mu = 3.986004418E5 # km**3/s**2
theta = np.radians(100)
r_p = 300 + 6378  # km
v_p = 15  # km/s

In [2]:
h = r_p * v_p # km**2/s
print(round(h, 2))

100170


To obtain $e$, we can use the orbit equation, evaluated at perigee ($\theta =$ 0°):

$$e = \frac{h^2}{\mu r} - 1$$

In [3]:
e = h**2 / (r_p * mu) - 1
print(round(e, 3))

2.77


Since $e > 1$, this trajectory is a hyperbola.

We should find the true anomaly of the asymptote, to ensure that our desired true anomaly is valid:

$$\theta_{\infty} = \cos^{-1}\left(-\frac{1}{e}\right)$$

In [4]:
theta_inf = np.arccos(-1 / e)
print(round(theta_inf, 3), f"{np.degrees(theta_inf):.3F}°")

1.94 111.166°


Therefore, our desired true anomaly is valid. Now, to find the time to fly to the true anomaly of 100°, we need to find $M_h$. $M_h$ is found from the hyperbolic eccentric anomaly, $F$, which is found from $\theta$:

$$F = 2 \tanh^{-1}\left(\sqrt{\frac{e - 1}{e + 1}}\tan\frac{\theta}{2}\right)$$

In [5]:
F_1 = 2 * np.arctanh(np.sqrt((e - 1)/(e + 1)) * np.tan(theta / 2)) % (2 * np.pi)
print(round(F_1, 3), f"{np.degrees(F_1):.3F}°")

2.293 131.359°


Like for the ellipse, to convert $F$ to the range $[0, 2\pi)$, we take the modulus with $2\pi$. In most programming languages, Python and MATLAB included, the `arctanh` function returns a value between $-\pi/2$ and $\pi/2$. When the result is multiplied by 2, it gives the range from $-\pi$ to $\pi$. We need to transform this angle to be in the range of $0$ to $2\pi$. To do so, we can take the **modulus** of the angle with $2\pi$.

The modulus is the remainder after division. In Python, the modulus operator is `%`, while in MATLAB, we have to use the function `mod(numerator, denominator)`. This works for both positive and negative numbers, and ensures that we get the correct angle for the appropriate quadrant.

Then, the mean anomaly is found from Kepler's equation:

$$M_h = e \sinh F - F$$

In [6]:
M_h1 = e * np.sinh(F_1) - F_1
print(round(M_h1, 3), f"{np.degrees(M_h1):.3F}°")

11.278 646.208°


Finally, calculating the time from the mean anomaly is done from the definition of the mean anomaly:

$$t = \frac{h^3}{\mu^2}\frac{1}{\left(e^2 - 1\right)^{3/2}} M_h$$

In [7]:
t_1 = h**3 / mu**2 * 1 / (e**2 - 1)**(3/2) * M_h1
print(t_1, t_1 / 3600)

4141.437375116706 1.1503992708657516


The total time is 4,141 seconds or just over 1 hour.

Now, let's calculate the true anomaly 3 hours later, after about 4 total hours since perigee have elapsed. Since we already have the orbital eccentricity and specific angular momentum, we can start by finding the mean anomaly at that time.

In [8]:
t_2 = 3 * 3600 + t_1  # sec
M_h2 = mu**2 / h**3 * (e**2 - 1)**(3/2) * t_2
print(round(t_2, 3), round(M_h2, 3))

14941.437 40.69


Now, we need to solve Kepler's equation to find the eccentric anomaly, $F$. Since the equation is transcendental in $F$, we need to use the Newton solver in SciPy. Since we know the derivative, we will define two Python functions:

1. Kepler's equation, $f(F) = 0$
2. The derivative of Kepler's equation with respect to $F$, $f'(F)$

In [9]:
def kepler(F, M_h, e):
    """Kepler's equation, to be used in a Newton solver."""
    return e * np.sinh(F) - F - M_h

def d_kepler_d_F(F, M_h, e):
    """The derivative of Kepler's equation, to be used in a Newton solver.
    
    Note that the argument M_h is unused, but must be present so the function
    arguments are consistent with the kepler function.
    """
    return e * np.cosh(F) - 1

F_2 = newton(func=kepler, fprime=d_kepler_d_F, x0=np.pi, args=(M_h2, e))
print(round(F_2, 3))

3.463


With this value for $F$, we can calculate the value for $\theta$. To avoid sign problems, we will use the tangent formula, rather than the cosine formula:

$$\tan\frac{\theta}{2} = \sqrt{\frac{e + 1}{e - 1}} \tanh\frac{F}{2}$$

or

$$\theta = 2\tan^{-1}\sqrt{\frac{e + 1}{e - 1}} \tanh\frac{F}{2}$$

Note here that we have to apply the modulus operator again.

In [10]:
theta_2 = (2 * np.arctan(np.sqrt((e + 1) / (e - 1)) * np.tanh(F_2 / 2))) % (2 * np.pi)
print(round(theta_2, 3), f"{np.degrees(theta_2):.3F}°")

1.881 107.780°


Then, the radius can be found from the orbit equation:

$$r = \frac{h^2}{\mu}\frac{1}{1 + e\cos\theta}$$

In [11]:
r_2 = h**2 / mu * 1 / (1 + e * np.cos(theta_2))
print(round(r_2, 0))

163180.0


The velocity components can be found from:

$$v_{\perp} = \frac{h}{r}$$

and

$$v_{r} = \frac{\mu}{h} e \sin\theta$$

and the speed is:

$$v = \sqrt{v_r^2 + v_{\perp}^2}$$

In [12]:
v_perp = h / r_2
v_r = mu / h * e * np.sin(theta_2)
v = np.sqrt(v_r**2 + v_perp**2)
print(round(v_perp, 3), round(v_r, 3), round(v, 3))

0.614 10.494 10.512


### MATLAB Solution

In MATLAB, the following code will give the same result:

```matlab
function kepler
    mu = 3.986e5; % km^3/s^2
    r_p = 300 + 6378; % km
    v_p = 15; % km/s
    h = r_p * v_p;
    e = h^2 / (mu * r_p) - 1;
    
    theta_1 = deg2rad(100);
    F_1 = 2 * atanh(sqrt((e - 1) / (e + 1)) * tan(theta_1 / 2));
    M_h1 = e * sinh(F_1) - F_1;
    t_1 = h^3 / mu^2 * 1 / (e^2 - 1)^(3 / 2) * M_h1;
    
    t_2 = t_1 + 3 * 3600;
    M_h2 = mu^2 / h^3 * (e^2 - 1)^(3 / 2) * t_2;

    function x = fun(F, M_h, e)
        x = e * sinh(F) - F - M_h;
    end

    F_2 = fzero(@(x) fun(x, M_h2, e), [3, 4]);
    t2 = 2 * atan(sqrt((e + 1) / (e - 1)) * tanh(F_2 / 2));
    theta_2 = mod(t2, 2 * pi);
    disp(rad2deg(theta_2))
    
    r_2 = h^2 / mu * 1 / (1 + e * cos(theta_2));
    v_perp = h / r_2;
    v_r = mu / h * e * sin(theta_2);
    v = sqrt(v_perp^2 + v_r^2);
    disp([v_perp, v_r, v])
end
```

We are using `fzero()` again to solve Kepler's equation. I'm not sure how sensitive `fzero()` will be to the initial guess.